# General procedure for rotating the stiffness tensor

Rotating a rank-4 tensor involves applying the rotation to each of the tensor’s indices. For a stiffness tensor $C_{ijkl}$ and a rotation matrix $R$, the transformation rule for rotating is given by (we use $C_{abcd}$ for the original tensor):

$$
C'_{ijkl} = R_{ia} R_{jb} R_{kc} R_{ld} C_{abcd}
$$

where $C'$ is the rotated stiffness tensor, $R$ is the rotation matrix, and the indices run over the dimensions (1 to 3). This operation is also known as a tensor contraction.

Here's a step-by-step process to rotate a 3x3x3x3 stiffness tensor using the `scipy.spatial.transform.Rotation` module to generate the rotation matrix:

In [1]:
import numpy as np
from scipy.spatial.transform import Rotation as R
from tensor_tools import _rearrange_tensor, _tensor_in_voigt

np.set_printoptions(suppress=True)


def rotate_stiffness_tensor(
    stiffness_tensor: np.ndarray, rotation_matrix: np.ndarray
) -> np.ndarray:
    """Rotate a 3x3x3x3 symmetric stiffness tensor using numpy.einsum.
    The operation is as follows:

    C'ijkl = Ria x Rjb x Rkc x Rld x Cabcd

    where Cabcd and C'ijkl are the original and the rotated tensor,
    respectively, and R the rotation matrix

    Parameters
    ----------
    stiffness_tensor : numpy.ndarray
        symmetric stiffness tensor

    rotation_matrix : numpy.ndarray
        3x3 rotation matrix

    Returns
    -------
    numpy.ndarray
        Rotated 3x3x3x3 symmetric stiffness tensor
    """
    # Ensure the inputs
    assert stiffness_tensor.shape == (3, 3, 3, 3), "Input tensor must be 3x3x3x3"
    assert rotation_matrix.shape == (3, 3), "Rotation matrix must be 3x3"

    rotated_tensor = np.einsum(
        "ia,jb,kc,ld,abcd->ijkl",
        rotation_matrix,
        rotation_matrix,
        rotation_matrix,
        rotation_matrix,
        stiffness_tensor,
    )

    return rotated_tensor


In [2]:
# elastic constants of San Carlos olivine at 1.5 GPa and 1027°C (1300 K), custom fitting from Zhang and Bass (2016)
C11 = 280.2
C22 = 182.1
C33 = 207.6
C44 =  56.8
C55 =  68.8
C66 =  68.5
C12 =  71.9
C13 =  67.2
C23 =  70.1

# Elastic stiffness tensor (in GPa) values as a Cij matrix
Cij = np.array(
    [[C11, C12, C13, 0.0, 0.0, 0.0],
    [ C12, C22, C23, 0.0, 0.0, 0.0],
    [ C13, C23, C33, 0.0, 0.0, 0.0],
    [ 0.0, 0.0, 0.0, C44, 0.0, 0.0],
    [ 0.0, 0.0, 0.0, 0.0, C55, 0.0],
    [ 0.0, 0.0, 0.0, 0.0, 0.0, C66]])

Cij

array([[280.2,  71.9,  67.2,   0. ,   0. ,   0. ],
       [ 71.9, 182.1,  70.1,   0. ,   0. ,   0. ],
       [ 67.2,  70.1, 207.6,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,  56.8,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,  68.8,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,  68.5]])

In [3]:
Cijkl = _rearrange_tensor(Cij)

In [4]:
# test
_tensor_in_voigt(Cijkl)

array([[280.2,  71.9,  67.2,   0. ,   0. ,   0. ],
       [ 71.9, 182.1,  70.1,   0. ,   0. ,   0. ],
       [ 67.2,  70.1, 207.6,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,  56.8,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,  68.8,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,  68.5]])

In [5]:
# Example usage:

# Define a rotation (for example, a 45-degree rotation around the z-axis)
rotation = R.from_euler('z', 45, degrees=True)
rotation_matrix = rotation.as_matrix()

rotated_tensor = rotate_stiffness_tensor(Cijkl, rotation_matrix)

Cij_rot = _tensor_in_voigt(rotated_tensor)

print("Rotated Stiffness Tensor:\n")
np.around(Cij_rot, 2)

Rotated Stiffness Tensor:



array([[220.02,  83.02,  68.65,   0.  ,   0.  ,  24.52],
       [ 83.02, 220.02,  68.65,   0.  ,   0.  ,  24.53],
       [ 68.65,  68.65, 207.6 ,   0.  ,   0.  ,  -1.45],
       [  0.  ,   0.  ,   0.  ,  62.8 ,   6.  ,   0.  ],
       [  0.  ,   0.  ,   0.  ,   6.  ,  62.8 ,   0.  ],
       [ 24.52,  24.53,  -1.45,   0.  ,   0.  ,  79.62]])

**MTEX result**:
```
tensor in Voigt matrix representation:
 220.02  83.02  68.65      0      0  24.53
  83.02 220.03  68.65      0      0  24.53
  68.65  68.65  207.6      0      0  -1.45
      0      0      0   62.8      6      0
      0      0      0      6   62.8      0
  24.52  24.53  -1.45      0      0  79.62
```

In [6]:
# Define a rotation (for example, a 45-degree rotation around the x-axis)
rotation = R.from_euler('x', 45, degrees=True)
rotation_matrix = rotation.as_matrix()

rotated_tensor = rotate_stiffness_tensor(Cijkl, rotation_matrix)

Cij_rot = _tensor_in_voigt(rotated_tensor)

print("Rotated Stiffness Tensor:")
np.around(Cij_rot, 2)

Rotated Stiffness Tensor:


array([[280.2 ,  69.55,  69.55,   2.35,   0.  ,   0.  ],
       [ 69.55, 189.28,  75.68,  -6.38,   0.  ,   0.  ],
       [ 69.55,  75.68, 189.27,  -6.37,   0.  ,   0.  ],
       [  2.35,  -6.38,  -6.37,  62.38,   0.  ,   0.  ],
       [  0.  ,   0.  ,   0.  ,   0.  ,  68.65,  -0.15],
       [  0.  ,   0.  ,   0.  ,   0.  ,  -0.15,  68.65]])

**MTEX result**:
```
tensor in Voigt matrix representation:
  280.2  69.55  69.55   2.35      0      0
  69.55 189.27  75.67  -6.38      0      0
  69.55  75.67 189.27  -6.38      0      0
   2.35  -6.38  -6.38  62.38      0      0
      0      0      0      0  68.65  -0.15
      0      0      0      0  -0.15  68.65
```

In [7]:
# Define a rotation (for example, a 45-degree rotation around the y-axis)
rotation = R.from_euler('y', 45, degrees=True)
rotation_matrix = rotation.as_matrix()

rotated_tensor = rotate_stiffness_tensor(Cijkl, rotation_matrix)

Cij_rot = _tensor_in_voigt(rotated_tensor)

print("Rotated Stiffness Tensor:")
np.around(Cij_rot, 2)

Rotated Stiffness Tensor:


array([[224.35,  71.  ,  86.75,   0.  , -18.15,   0.  ],
       [ 71.  , 182.1 ,  71.  ,   0.  ,  -0.9 ,   0.  ],
       [ 86.75,  71.  , 224.35,   0.  , -18.15,   0.  ],
       [  0.  ,   0.  ,   0.  ,  62.65,   0.  ,  -5.85],
       [-18.15,  -0.9 , -18.15,   0.  ,  88.35,   0.  ],
       [  0.  ,   0.  ,   0.  ,  -5.85,   0.  ,  62.65]])

**MTEX result**:
```
tensor in Voigt matrix representation:
 224.35     71  86.75      0 -18.15      0
     71  182.1     71      0   -0.9      0
  86.75     71 224.35      0 -18.15      0
      0      0      0  62.65      0  -5.85
 -18.15   -0.9 -18.15      0  88.35      0
      0      0      0  -5.85      0  62.65
```

MTEX pole figure plot

![90 rot](https://raw.githubusercontent.com/marcoalopez/PyRockWave/main/src/img/mtex_ol_rotated.png)

___

In [8]:
# Define a rotation (for example, a 90-degree rotation around the z-axis)
rotation = R.from_euler('z', 90, degrees=True)
rotated_tensor = rotate_stiffness_tensor(Cijkl, rotation.as_matrix())
Cij_rot = _tensor_in_voigt(rotated_tensor)

print("Rotated Stiffness Tensor:") 
np.around(Cij_rot, 2)

Rotated Stiffness Tensor:


array([[182.1,  71.9,  70.1,   0. ,   0. ,   0. ],
       [ 71.9, 280.2,  67.2,   0. ,   0. ,   0. ],
       [ 70.1,  67.2, 207.6,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,  68.8,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,  56.8,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,  68.5]])

**MTEX result**:
```
 tensor in Voigt matrix representation:
 182.1  71.9  70.1     0     0     0
  71.9 280.2  67.2     0     0     0
  70.1  67.2 207.6     0     0     0
     0     0     0  68.8     0     0
     0     0     0     0  56.8     0
     0     0     0     0     0  68.5
```

In [9]:
# Define a rotation (for example, a 90-degree rotation around the x-axis)
rotation = R.from_euler('x', 90, degrees=True)
rotated_tensor = rotate_stiffness_tensor(Cijkl, rotation.as_matrix())
Cij_rot = _tensor_in_voigt(rotated_tensor)

print("Rotated Stiffness Tensor:") 
np.around(Cij_rot, 2)

Rotated Stiffness Tensor:


array([[280.2,  67.2,  71.9,   0. ,   0. ,   0. ],
       [ 67.2, 207.6,  70.1,   0. ,   0. ,   0. ],
       [ 71.9,  70.1, 182.1,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,  56.8,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,  68.5,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,  68.8]])

**MTEX result**:
```
tensor in Voigt matrix representation:
 280.2  67.2  71.9     0     0     0
  67.2 207.6  70.1     0     0     0
  71.9  70.1 182.1     0     0     0
     0     0     0  56.8     0     0
     0     0     0     0  68.5     0
     0     0     0     0     0  68.8
```

In [10]:
# Define a rotation (for example, a 90-degree rotation around the y-axis)
rotation = R.from_euler('y', 90, degrees=True)
rotated_tensor = rotate_stiffness_tensor(Cijkl, rotation.as_matrix())
Cij_rot = _tensor_in_voigt(rotated_tensor)

print("Rotated Stiffness Tensor:") 
np.around(Cij_rot, 2)

Rotated Stiffness Tensor:


array([[207.6,  70.1,  67.2,   0. ,   0. ,   0. ],
       [ 70.1, 182.1,  71.9,   0. ,   0. ,   0. ],
       [ 67.2,  71.9, 280.2,   0. ,   0. ,   0. ],
       [  0. ,   0. ,   0. ,  68.5,   0. ,   0. ],
       [  0. ,   0. ,   0. ,   0. ,  68.8,   0. ],
       [  0. ,   0. ,   0. ,   0. ,   0. ,  56.8]])

**MTEX result**:
```
tensor in Voigt matrix representation:
 207.6  70.1  67.2     0     0     0
  70.1 182.1  71.9     0     0     0
  67.2  71.9 280.2     0     0     0
     0     0     0  68.5     0     0
     0     0     0     0  68.8     0
     0     0     0     0     0  56.8
```

MTEX pole figure plot

![90 rot](https://raw.githubusercontent.com/marcoalopez/PyRockWave/main/src/img/mtex_ol.png)